In [1]:
import warnings
warnings.filterwarnings('ignore')

# 앙상블(Ensenble)
- 여러 개의 분류 모델을 조합해서 더 나은 성능을 내는 방법
- Decision Tree 모델을 이용한 랜덤포레스트가 대표적임

---
# 랜덤포레스트(Random Forest)
- 랜덤포레스트는 부트스트랩 샘플을 사용한다.
- 부트스트랩 샘플링은 중복을 허용하는 샘플링 방법이다.

In [2]:
import pandas as pd
wine = pd.read_csv('data/wine.csv',)
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [5]:
data = wine.drop('class', axis=1)
target = wine['class']

In [7]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [12]:
# import
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# model
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
# n_jobs: 병렬처리 수: -1은 모든 코어를 사용하는 것을 의미한다.

# scores
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print('train_score:', np.mean(scores['train_score']))
print('test_score:', np.mean(scores['test_score']))

train_score: 0.9973541965122431
test_score: 0.8905151032797809


In [10]:
rf.fit(train_input, train_target)
# Feature 중요도 확인
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [13]:
# 부트스트랩 결정 시 남은 생플(oob: out of bag)로도 특성을 구분할 수 있다.
rf = RandomForestClassifier(n_jobs=-1, random_state=42, oob_score=True)
rf.fit(train_input, train_target)
print('oob_score:', rf.oob_score_)


oob_score: 0.8934000384837406


---
# Extra Tree
- 기본적으로 100개의 트리를 사용
- 노드 분할 시 특성의 제곱근의 갯수를 사용
- 특성을 랜덤하게 선택한다. (Sugar를 뺄 수도 있음)
- 속도는 랜덤포레스트보다 빠르다(특성을 막 선택해서)

In [14]:
# extra tree import
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

# 주요 Feature 확인을 위해 fit 해봄
et.fit(train_input, train_target)

# 주요 feature 확인
print(et.feature_importances_)

# scores 계산해 출력
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print('train_score:', np.mean(scores['train_score']))
print('test_score:', np.mean(scores['test_score']))

[0.20183568 0.52242907 0.27573525]
train_score: 0.9974503966084433
test_score: 0.8887848893166506


---
# Gradient Boosting(그래디언트 부스팅)
- 가장 유명한 알고리즘 중 하나.
- 경사하강법처럼 손실함수를 사용한다.
- 손실함수를 보고 트리를 추가하여 최적값을 도출한다.
- Decision Tree Regressor를 사용하여 손실함수를 계산하고이를 계속 낮추기 위해 트리를 추가한다.(Max Depth가 추가됨)
- 경사를 이동하면서 경사의 이동거리를 제어하는 learning-rate(기본:0.1)를 사용한다.
- max_depth를 3으로 제어하여 깊이가 낮으므로 과대적합을 방지할 수 있다.
- 단점은 손실함수를 보고 트리를 추가하면서 진행하기 때문에 병렬처리(n_jobs)를 할 수 없다.
(한꺼번에 막 트리를 만드는게 아니라 하나 하고 판단하고 하나 하고 판단하고...)

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=42)

# scores 계산해 출력
scores = cross_validate(gbc, train_input, train_target, return_train_score=True)
print('train_score:', np.mean(scores['train_score']))
print('test_score:', np.mean(scores['test_score']))

train_score: 0.8881086892152563
test_score: 0.8720430147331015


In [16]:
# 기본값 변경해 실험: tree 500개, learning rate 0.2로
gbc = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gbc, train_input, train_target, return_train_score=True)
print('train_score:', np.mean(scores['train_score']))
print('test_score:', np.mean(scores['test_score']))

train_score: 0.9464595437171814
test_score: 0.8780082549788999


---
# 히스토그램 기반 그래디언트 부스팅(Histogram gradient Boosting)
- 훈련데이터를 256개의 구간으로 나누어서 훈련시키는 방법
- 특성의 범위가 제한되어 빠른 속도를 제공한다.
- 제한된 구간이므로 과대적합을 방지한다
- 아직은 실험단계인 모델

In [26]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgbc = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgbc, train_input, train_target, return_train_score=True, n_jobs=-1)
print('train_score:', np.mean(scores['train_score']))
print('test_score:', np.mean(scores['test_score']))

train_score: 0.9321723946453317
test_score: 0.8801241948619236


# * 번외 기능 *
### Permutation Importance(치환 중요도) - RDE처럼 중요 Column 찾는 것
- 어떤 특성이 중요한지를 알아내는 방법
- 각 특성(Feature)별 샘플을 섞어서 계산을 한 후, 원래 sample들과의 차이를 계산해서 차이가 많이 나는 Feature가 중요하다는 판단
- 예를 들어 Sugar Data가 막 섞이면 차이가 더 심해질 것.

In [30]:
from sklearn.inspection import permutation_importance

# Train data를 가지고 확인

# hgbc 말고 아무 모델이라도 무관
hgbc.fit(train_input, train_target)
result = permutation_importance(hgbc, train_input, train_target, n_repeats=10, random_state=42)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


### 해석
- Sugar의 Sample을 무작위로 섞으면 23%만큼 정확도가 떨어진다
- 나머지들은 막 섞어도 정확도가 8%정도밖에 안 떨어진다
- 정확도에 가장 큰 영향을 미치는 것은 Sugar다 - 주요 특성은 Sugar

In [31]:
# Test data를 가지고 확인
hgbc.fit(train_input, train_target)
result = permutation_importance(hgbc, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


---
# XGBoost
- kaggle에서 제일 많이 사용

In [35]:
#!pip install xgboost

In [36]:
# xgboost import
from xgboost import XGBClassifier

In [41]:
xgbc = XGBClassifier(
  random_state=42, 
  tree_method='hist',  # 히스토그램 그레디언트 부스팅
  use_label_encoder=False, 
  n_jobs=-1, 
  eval_metric='logloss' # 로지스틱 손실함수 사용
)
scores = cross_validate(xgbc, train_input, train_target, return_train_score=True, n_jobs=-1)
print('train_score:', np.mean(scores['train_score']))
print('test_score:', np.mean(scores['test_score']))

train_score: 0.9555033709953124
test_score: 0.8799326275264677


---
# LightGBM

In [50]:
from lightgbm import LGBMClassifier

In [51]:
lgbc = LGBMClassifier(random_state=42)

scores = cross_validate(lgbc, train_input, train_target, return_train_score=True, n_jobs=-1)
print('train_score:', np.mean(scores['train_score']))
print('test_score:', np.mean(scores['test_score']))

train_score: 0.935828414851749
test_score: 0.8801251203079884


---
# 앙상블 정리
- 앙상블 학습은 정형데이터에서 가장 뛰어난 성능을 내는 머신러닝 알고리즘 중의 하나
### 랜덤포레스트
- 부트스트랩 사용, 대표 앙상블 학습 알고리즘
### 엑스트라 트리
- 결정트리의 노드를 랜덤하게 분할
### 그래디언트 부스팅
- 결정트리의 손실을 보완하는 식으로 얕은 결정트리를 연속해 추가해 사용
### 히스토그램 기반 그래디언트 부스팅
- 훈련데이터를 256개 정수 구간으로 나누어 빠르고 높은 성능을 냄
### 기타
- XGBoost
- lightGBM